In [1]:
import sys
sys.path.append('..')
from typing import Dict, List
from copy import deepcopy

In [2]:
import pandas as pd
from mars.movement import *
from mars.tool import *
import numpy as np

In [3]:
from mars.action import Action, ActionType
import mars.serializer as ms
from collections import namedtuple

In [4]:
from pymongo import MongoClient
from bson import ObjectId

In [5]:

def get_point_from_row(df_row_data):

    vector = np.array([
        df_row_data.x_j1,
        df_row_data.y_j2,
        df_row_data.z_j3,
        df_row_data.w_j4,
        df_row_data.p_j5,
        df_row_data.r_j6
    ])

    e1 = df_row_data.e1
    speed = df_row_data.speed
    cnt = df_row_data.cnt
    path = df_row_data.path
    pos_type = df_row_data.position_type

    if pos_type == "JOINT":
        position = PositionJoint(vector, e1)
    elif pos_type == "CRT":
        wrist = WristConfig.FLIP if df_row_data.wrist =='F' else WristConfig.NOFLIP
        forearm = ForeArmConfig.UP if df_row_data.forearm == 'U' else ForeArmConfig.DOWN
        arm = ArmConfig.TOWARD if df_row_data.arm =='T' else ArmConfig.BACKWARD

        config = Configuration(wrist, forearm, arm)
        position = PositionCrt(vector, e1, config)
    else:
        raise Exception("Parsing default : {pos_type} not valid.")

    path = Path[df_row_data.path]

    return Point(cnt, speed, path, position)


"""
def serialize_for_mongo(action, up_dependences_ids, down_dependences_ids):
    
    action_doc = action.to_dict()
    del(action_doc['id'])

    if up_dependences_ids:
        action_doc['UPSTREAM_dependences'] = up_dependences_ids

    if down_dependences_ids:
        action_doc['DOWNSTREAM_dependences'] = down_dependences_ids

    return action_doc
"""

def save_in_mongo(action_dict: Dict or List[Dict]):
    
    c = MongoClient()
    carrier = c.get_database('mars').get_collection('carrier')

    if type(action_dict) == list:
        res = carrier.insert_many(action_dict)
    else:
        res = carrier.insert_one(action_dict)

    if not res.acknowledged :
        raise Exception("Error during insertion")
        
    else :
        if type(action_dict) == list:
            # print('insertion success, document id : ', str(res.inserted_ids))
            return res.inserted_ids
        else :
            # print('insertion success, document id : ', str(res.inserted_id))
            return res.inserted_id

    """
    if type(action) == list:
        actions_docs = []
        for a in action:
            if type(a) == tuple:
                action_doc = serialize_for_mongo(a[0], up_dependences_ids, down_dependences_ids)
                action_doc.update(a[1])
            else:
                action_doc = serialize_for_mongo(a, up_dependences_ids, down_dependences_ids)
            
            actions_docs.append(action_doc)

        res = carrier.insert_many(actions_docs)

        if not res.acknowledged :
            print('Exception during insertion')
            return None
        else :
            print('insertion success, document id : ', str(res.inserted_ids))
            return res.inserted_ids
    
    else:
        if type(action) == tuple:
            action_doc = serialize_for_mongo(action[0], up_dependences_ids, down_dependences_ids)
            action_doc.update(action[1])
        else:
            action_doc = serialize_for_mongo(action, up_dependences_ids, down_dependences_ids)
""" 

def get_action_type(action: str) :
    if(action == 'station'):
        return ActionType['MOVE']['STATION']['WORK']
    elif (action == 'work'):
        return ActionType['MOVE']['ARM']['WORK']
    elif (action == 'approach'):
        return ActionType['MOVE']['ARM']['APPROACH']
    elif (action == 'clearance'):
        return ActionType['MOVE']['ARM']['CLEARANCE']
    elif (action == 'home'):
        return ActionType['MOVE']['STATION']['HOME']
    elif (action == 'tool'):
        return ActionType['MOVE']['STATION']['TOOL']
    else:
        raise Exception("default on action type")

def get_action_from_df(action_type: str,
                       description: str,
                       dataframe: pd.DataFrame) -> Action :
    points: List[Point] = []

    for line in dataframe.iterrows():
        points.append(get_point_from_row(line[1]))
    
    ut_uf = dataframe[['UT','UF', 'work_order']].drop_duplicates()
    ut, uf, work_order = int(ut_uf.iloc[0].UT),\
                         int(ut_uf.iloc[0].UF),\
                         int(ut_uf.iloc[0].work_order) if not pd.isna(ut_uf.iloc[0].work_order) else None

    movement = Movement(uf, ut, points)
    
    return Action(id='',
                  atype = get_action_type(action_type),
                  definition = movement,
                  description = description,
                  work_order=work_order)

def get_additionnal_info(dataframe: pd.DataFrame, infolist: List[str]):
    add_info_df = dataframe[infolist].drop_duplicates()
    add_list = [(name, add_info_df.iloc[0][name]) for name in infolist]
    add_list = [(name, convert_np(val)) for name, val in add_list]
        
    return dict(add_list)

def convert_np(value):
    if type(value) == np.int64:
        return int(value)
    elif type(value) == np.float64:
        return float(value)
    else:
        return value
    

In [6]:
df = pd.read_csv('./data/C35_web.csv')

In [7]:
# save global actions

# go home action
df_home = df[df.mvt =='home']
go_home = get_action_from_df('home', 'go on home position', df_home)
go_home = go_home.to_dict()
go_home['_id'] = 'home'
go_home_id = save_in_mongo(go_home)

# go load tool position action
df_tool = df[df.mvt =="tool"]
go_tool = get_action_from_df('tool', 'go to load/unload tool position', df_tool)
go_tool = go_tool.to_dict()
go_tool['_id'] = 'load_tool_position'
go_tool_id = save_in_mongo(go_tool)

In [8]:
# C35 WEB actions

# get information from pandas dataframe
df_station = df[df.mvt == "station"]
df_approach = df[df.mvt == "approach"]
df_clearance = df[df.mvt == "clearance"]
df_work = df[df.mvt == "work"]

# list of all rails
rails = [1, 2, 3, 4, 5, 6]

# unload effector action
ulte_def = UnloadManipulation(3, 3, "effector", "WEBEFFECTOR")
unload_eff = Action('ult', ActionType['UNLOAD']['EFFECTOR'], ulte_def, "unload web effector")
unload_eff = unload_eff.to_dict()
del unload_eff['_id']
unload_id = save_in_mongo(unload_eff)

# load effector action
lte_def = LoadManipulation(1, 3, "effector", "WEBEFFECTOR")
load_eff= Action('lt', ActionType['LOAD']['EFFECTOR'], lte_def, "load web effector")
load_eff = load_eff.to_dict()
del load_eff['_id']

# add a dependence for load effector => unload effector 
load_eff['dependences'] = [{'action': unload_id, 'type': 'DOWNSTREAM'}]
load_id = save_in_mongo(load_eff)


'''
# load drill tool
ltd_def = LoadManipulation(3,1,"drill", "4.1_DRILL")
load_drill = Action('lt', ActionType['LOAD']['TOOL'], ltd_def, "load drill tool")

# unload drill tool
ultd_def = UnloadManipulation(3,1,"drill", "4.1_DRILL")
unload_eff = Action('ult', ActionType['UNLOAD']['TOOL'], ulte_def, "unload drill tool")
'''

targeted_area = {
    'area':'WEB'
  }

# for each rail
for rail in rails: 

  # -- STATION --
  # prepare and insert station mvt in mongo
  df_temp = df_station[df_station.rail==rail]
  station = get_action_from_df('station',
                              'station movement on rail {rail}'.format(rail=rail),
                              df_temp)
  station = station.to_dict()
  add_info = get_additionnal_info(df_temp, ['designation', 'reference', 'id'])
  del station['_id']
  # add station product reference
  station['product_reference'] = {
    "designation": add_info['designation'],
    "reference": add_info['reference'],
    "id": add_info['id'],
  }

  station['targeted_area'] = targeted_area

  # save station action in mongo
  station_id = save_in_mongo(station)
  
  # -- CLEARANCE --
  # prepare and insert clearance mvt in mongo
  df_temp = df_clearance[df_clearance.rail==rail]
  clearance = get_action_from_df('clearance',
                              'clearance movement from rail {rail}'.format(rail=rail),
                              df_temp)
  clearance = clearance.to_dict()
  del clearance['_id']
  # same product reference as station
  clearance['product_reference'] = deepcopy(station['product_reference'])
  clearance['targeted_area'] = targeted_area
  # save in mongo
  clearance_id = save_in_mongo(clearance)

  # -- APPROACH --
  # prepare and insert approach mvt in mongo
  df_temp = df_approach[df_approach.rail==rail]
  approach = get_action_from_df('approach',
                              'approach movement to rail {rail}'.format(rail=rail),
                              df_temp)

  approach = approach.to_dict()
  del approach['_id']
  approach['product_reference'] = deepcopy(station['product_reference'])
  approach['targeted_area'] = targeted_area
  # add dependences => load effector and move station as UPSTREAM, clearance as DOWNSTREAM
  approach['dependences'] = [{'action':load_id, 'type': 'UPSTREAM'},
                             {'action':station_id, 'type': 'UPSTREAM'},
                             {'action':clearance_id, 'type': 'DOWNSTREAM'}]
  #save in mongo
  approach_id = save_in_mongo(approach)

  # -- WORK MOVEMENTS
  # prepare and insert work mvt in mongo
  df_temp = df_work[df_work.rail==rail]
  w_actions = []
  id_list = [item[1] for item in df_temp.id.iteritems()]

  for id in id_list :
    df_w = df_temp[df_temp.id == id]
    add_info = get_additionnal_info(df_w, ['designation', 'reference', 'id'])
    wa = get_action_from_df('work',
        'work movement on location of fastener {ref}.{id} of rail {rail}'
        .format(ref=add_info['reference'], id=add_info['id'], rail=rail),
        df_w)
    wa = wa.to_dict()
    del wa['_id']
    wa['product_reference'] = {
      "designation": add_info['designation'],
      "reference": add_info['reference'], 
      "id": add_info['id'],
      "parent": deepcopy(station['product_reference'])
    }
    
    wa['targeted_area'] = targeted_area

    # add dependences => approach as UPSTREAM
    wa['dependences'] = [{'action':approach_id, 'type': 'UPSTREAM'}]
    w_actions.append(wa)

  # save the list of work movement in mongo
  work_ids = save_in_mongo(w_actions)

In [9]:
df = pd.read_csv('./data/C35_flange.csv')

In [10]:
# get information from pandas dataframe
df_station = df[df.mvt == "station"]
df_approach = df[df.mvt == "approach"]
df_clearance = df[df.mvt == "clearance"]
df_work = df[df.mvt == "work"]

# list of all rails
rails = [1, 2, 3, 4, 5, 6]
sides = ['AV_D', 'AV_G', 'AR_D', 'AR_G']

# unload effector action
ulte_def = UnloadManipulation(3, 3, "effector", "FLANGEEFFECTOR")
unload_eff = Action('ult', ActionType['UNLOAD']['EFFECTOR'], ulte_def, "unload flange effector")
unload_eff = unload_eff.to_dict()
del unload_eff['_id']
unload_id = save_in_mongo(unload_eff)

# load effector action
lte_def = LoadManipulation(2, 3, "effector", "FLANGEEFFECTOR")
load_eff= Action('lt', ActionType['LOAD']['EFFECTOR'], lte_def, "load flange effector")
load_eff = load_eff.to_dict()
del load_eff['_id']

# add a dependence for load effector => unload effector 
load_eff['dependences'] = [{'action': unload_id, 'type': 'DOWNSTREAM'}]
load_id = save_in_mongo(load_eff)
 
for rail in rails:
  for side in sides:

    df_temp = df_station[(df_station.localisation == side) & (df_station.rail == rail)]
    
    add_info = get_additionnal_info(df_temp, ['designation', 'reference', 'id', 'pos_x', 'pos_y'])
    station = get_action_from_df('station',
                              'station movement on rail {rail} at {pos_x}-{pos_y} position'.format(rail=rail, pos_x = add_info['pos_x'], pos_y=add_info['pos_y']),
                              df_temp)
    station = station.to_dict()
    
    del station['_id']
    # add station product reference
    station['product_reference'] = {
      "designation": add_info['designation'],
      "reference": add_info['reference'],
      "id": add_info['id'],
    }

    station['targeted_area'] = {
      'area': 'FLANGE',
      'side': add_info['pos_y'],
      'position': add_info['pos_x']
    }
    
    # save station action in mongo
    station_id = save_in_mongo(station)
    
    # -- CLEARANCE --
    # prepare and insert clearance mvt in mongo
    df_temp = df_clearance[(df_clearance.rail==rail) & (df_clearance.localisation == side)]

    # addinfo : same as station
    clearance = get_action_from_df('clearance',
                                'clearance movement from rail {rail} at {pos_x}-{pos_y} position'.format(rail=rail, pos_x = add_info['pos_x'], pos_y=add_info['pos_y']),
                                df_temp)
    clearance = clearance.to_dict()
    del clearance['_id']
    # same product reference as station
    clearance['product_reference'] = deepcopy(station['product_reference'])
    clearance['targeted_area'] = deepcopy(station['targeted_area'])
    # save in mongo
    clearance_id = save_in_mongo(clearance)

    # -- APPROACH --
    # prepare and insert approach mvt in mongo
    df_temp = df_approach[(df_approach.rail==rail) & (df_approach.localisation == side)]
    # addinfo : same as station
    approach = get_action_from_df('approach',
                                'approach movement to rail {rail} at {pos_x}-{pos_y} position'.format(rail=rail, pos_x = add_info['pos_x'], pos_y=add_info['pos_y']),
                                df_temp)

    approach = approach.to_dict()
    del approach['_id']
    approach['product_reference'] = deepcopy(station['product_reference'])
    approach['targeted_area'] = deepcopy(station['targeted_area'])
    # add dependences => load effector and move station as UPSTREAM, clearance as DOWNSTREAM
    approach['dependences'] = [{'action':load_id, 'type': 'UPSTREAM'},
                              {'action':station_id, 'type': 'UPSTREAM'},
                              {'action':clearance_id, 'type': 'DOWNSTREAM'}]
    #save in mongo
    approach_id = save_in_mongo(approach)

    # -- WORK MOVEMENTS
    # prepare and insert work mvt in mongo
    df_temp = df_work[(df_work.rail==rail) & (df_work.localisation == side)]
    w_actions = []
    id_list = [item[1] for item in df_temp.id.iteritems()]

    for id in id_list :
      df_w = df_temp[df_temp.id == id]
      add_info = get_additionnal_info(df_w, ['designation', 'reference', 'id', 'pos_x', 'pos_y'])
      wa = get_action_from_df('work',
          'work movement on location of fastener {ref}.{id} of rail {rail} at {pos_x}-{pos_y} position'
          .format(ref=add_info['reference'], id=add_info['id'], rail=rail, pos_x = add_info['pos_x'], pos_y=add_info['pos_y']),
          df_w)
      wa = wa.to_dict()
      del wa['_id']
      wa['product_reference'] = {
        "designation": add_info['designation'],
        "reference": add_info['reference'], 
        "id": add_info['id'],
        "parent": deepcopy(station['product_reference'])
      }
      
      wa['targeted_area'] = deepcopy(station['targeted_area'])

      # add dependences => approach as UPSTREAM
      wa['dependences'] = [{'action':approach_id, 'type': 'UPSTREAM'}]
      w_actions.append(wa)

    # save the list of work movement in mongo
    work_ids = save_in_mongo(w_actions)
  